# Création des tables

In [1]:
!pip install chess
import os
import torch
import chess
import chess.pgn
from tqdm import tqdm

from board_utils import *
from mcts import *
from move_tables import * 
from neural_network import *
from self_play import *
from training import *

## Tables pour la conversion de move vers index (et inversement)

Conversion move <-> index

In [2]:
move_to_index, index_to_move = create_tables()
num_moves = len(index_to_move)
print(f"\n=> Nombre de coups total : {num_moves}\n")

Nombre total de coups stockés = 1840 (attendu ~1840).
Détails :
  Dames (queen-like) : 1456
  Cavaliers          : 336
  Pions (dont promos): 44
  Rois               : 0
  Roques             : 4

=> Nombre de coups total : 1840



## Fonction pour convertir une position sous format echec (position chess.move) vers s (un tensor pour le réseau)

s est donc de taille [119, 8, 8] en théorie selon alpha zéro mais nous on prend [43, 8, 8]
Chaque dimension de 119 est une pièce (6 * 2), le 13 encode à qui est le tour, les 4 autres pour savoir si on a le droit de roquer ou non, les 26 suivants pour l'historique des deux positions précedentes (car 13 * 2 = 26)

Donc en tout s (state) est de dimension : encode_board(chess.board) = s  (s.shape = [43, 8, 8])

Rem : Chaque canal (ex à qui est le tour) n'a que des valeurs 1 si c'est au blanc, -1 sinon (donc un tensor  8 * 8)

In [3]:
# Un exemplle d'encodage pour comprendre
board = chess.Board()  # Initialiser un échiquier standard

# Simuler quelques mouvements pour créer un historique
history = []
history.append(board.copy())  # Position initiale
board.push_san("e4")
history.append(board.copy())  # Après 1.e4
board.push_san("e5")

# Encoder l'échiquier avec l'historique des 2 positions précédentes
encoded = encode_board(board, history=history)

print("Entrée : ")
print(board)
print("Sortie : ")
print(encoded.shape)  # torch.Size([43, 8, 8])
print(encoded)

Entrée : 
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Sortie : 
torch.Size([43, 8, 8])
tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  1.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ..., 

## Hyperparamètres

In [10]:
dataset_path = '/kaggle/working/'
os.makedirs(dataset_path, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

# Hyperparamètres
MCTS_SIMS = 800          # Nombre de simulations MCTS
NUM_ITER = 10           # Nombre d’itérations de boucle d'entraînement
GAMES_PER_ITER = 63     # Nombre de parties self-play par itération
MAX_BUFFER_SIZE = 100000
LR = 0.2                 # Taux d'apprentissage initial
EPOCHS_PER_ITER = 6
BATCH_SIZE = 1200    
TEMPERATURE_MOVES = 30
MAX_MOVES_GAME = 150

Device: cpu


### Création du réseau de neuronne 

In [11]:
net = AlphaZeroNet(in_channels=43, channels=128, num_blocks=8, num_moves=num_moves)
replay_buffer = []

# Nombre de paramètres
print('Nombre de paramètres entraînables :', sum(p.numel() for p in net.parameters() if p.requires_grad))

Nombre de paramètres entraînables : 17655348


## Boucle principale 

In [ ]:
net_versions = []      # Pour sauvegarder différentes versions du réseau
net_versions.append(net.state_dict()) 

for iteration in range(NUM_ITER):
    print("\n" + "="*60)
    print(f"Iteration {iteration+1}/{NUM_ITER}")

    # 1) Self-play : récolter des données
    new_data = []
    for _ in tqdm(range(GAMES_PER_ITER)):
        game_data = play_one_game(net,
                                    device=device,
                                    max_moves=MAX_MOVES_GAME,
                                    mcts_sims=MCTS_SIMS,
                                    c_puct=1.0,
                                    temperature_moves=TEMPERATURE_MOVES,
                                    index_to_move = index_to_move,
                                    move_to_index = move_to_index,
                                    num_moves = num_moves)
        new_data.extend(game_data)
    print(f"  => {len(new_data)} positions collectées.")

    # 2) Replay buffer
    replay_buffer.extend(new_data)
    if len(replay_buffer) > MAX_BUFFER_SIZE:
        replay_buffer = replay_buffer[-MAX_BUFFER_SIZE:]
    print(f"  => Taille du replay buffer = {len(replay_buffer)}")

    # 3) Entraînement
    print("  => Entraînement...")
    avg_loss = train_on_data(net,
                                replay_buffer,
                                batch_size=BATCH_SIZE,
                                lr=LR,
                                epochs=EPOCHS_PER_ITER,
                                device=device)
    print(f"  => Loss moyen : {avg_loss:.4f}")

    # On stocke la nouvelle version du réseau
    net_versions.append(net.state_dict())
    model_path = f"alphazero_iteration_{iteration+1}.pth"
    torch.save(net.state_dict(), model_path)
    print(f"  => Modèle sauvegardé dans '{model_path}'")

    # 4) Comparaison du modèle vs sa version précédente toutes les 4 itérations
    if iteration % 4 == 1:
        if len(net_versions) >= 2:
            print("  => Évaluation de la nouvelle version contre la précédente...")
            net_prev = AlphaZeroNet(in_channels=43, channels=128, num_blocks=8, num_moves=num_moves)
            net_prev.load_state_dict(net_versions[-2])
            net_prev.to(device)

            net_current = AlphaZeroNet(in_channels=43, channels=128, num_blocks=8, num_moves=num_moves)
            net_current.load_state_dict(net_versions[-1])
            net_current.to(device)

            nb_eval_games = 2
            wins_current, draws, wins_prev = play_match(net_current, net_prev,
                                                        nb_games=nb_eval_games,
                                                        device=device,
                                                        max_moves=MAX_MOVES_GAME,
                                                        mcts_sims=100,
                                                        c_puct=1.0,
                                                        index_to_move = index_to_move,
                                                        move_to_index = move_to_index,
                                                        num_moves = num_moves)
            print(f"    Résultats (réseau actuel vs réseau précédent) sur {nb_eval_games} parties :")
            print(f"      Victoires (actuel) = {wins_current}")
            print(f"      Nulles             = {draws}")
            print(f"      Victoires (ancien) = {wins_prev}")

print("\nEntraînement terminé !")


Iteration 1/20


  0%|          | 0/63 [00:00<?, ?it/s]

  2%|▏         | 1/63 [01:24<1:27:12, 84.39s/it]


KeyboardInterrupt: 